In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import joblib
import datasets

# Train

In [2]:
import re

def remove_image_string(input_string):
    pattern = r"!\[(.*?)\]\(.*?\)\{width=\".*?\" height=\".*?\"\}|!\[.*?\]\(.*?\)|\[.*?\]\{.*?\}"
    result = re.sub(pattern, "", input_string)
    return result

def is_all_chinese_or_english(text):
    for char in text:
        if not is_chinese_or_english(char):
            return False
    return True

def is_chinese_or_english(char):
    # 判断是否为中文
    if '\u4e00' <= char <= '\u9fff':
        return True
    # 判断是否为英文
    elif 'a' <= char.lower() <= 'z':
        return True
    else:
        return False

def remove_noise_character(input_string):
    pattern = r"[>*|image|data|media|png]"
    
    return re.sub(pattern, "", input_string)

def one_text_pre_process(text):
    precessed_text_split_lines = []
    
    for line in text.splitlines():
        remove_image_line = remove_image_string(line)

        if remove_image_line.strip() in ["", ">"]:
            continue

        remove_image_line = remove_noise_character(remove_image_line)
        precessed_text_split_lines.append(remove_image_line)
        
    return "\n".join(precessed_text_split_lines)

def pre_process(text_list):
    precessed_text_list = [one_text_pre_process(text) for text in text_list]
    return precessed_text_list

def dataset_map_pre_process(row):
    row["text"] = one_text_pre_process(row["text"])
    return row

In [3]:
import jieba


def chinese_tokenizer(text):
    tokens = jieba.cut(text)
    return list(filter(lambda x:is_all_chinese_or_english(x), tokens))


def train(dataset):
    vectorizer = CountVectorizer(tokenizer=chinese_tokenizer, ngram_range=(1, 2))
    classifier = LogisticRegression(max_iter=100, verbose=1)
    model = make_pipeline(vectorizer, classifier)
    
    model.fit(dataset["text"], dataset["label"])
    return model

In [117]:
# datadict = datasets.load_dataset("ranWang/test_paper_textClassifier")
datadict = datasets.load_from_disk("./new_classification_datasets")
datadict

DatasetDict({
    test: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 387
    })
    train: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 10191
    })
})

In [118]:
train_dataset = datadict["train"]
train_dataset = train_dataset.map(dataset_map_pre_process)

Loading cached processed dataset at /home/ran/WorkSpace/my_Exam-Question-Bank-Dataset-zh_mnbvc/notebook/new_classification_datasets/train/cache-234ddfbed942625b.arrow


In [119]:
# 随机打乱
# shuffle(42) Accuracy = 0.978
# shuffle(100) Accuracy = 0.981
train_dataset = train_dataset.shuffle(36)
# 验证集大小
validation_size = int(len(train_dataset)/10)
validation_dataset = train_dataset.select(range(validation_size))
train_dataset = train_dataset.select(range(validation_size, len(train_dataset)))

In [120]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 9172
})

In [121]:
model = train(train_dataset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      6687569     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.35755D+03    |proj g|=  1.46310D+05

At iterate   50    f=  1.27083D+02    |proj g|=  3.05136D+02

At iterate  100    f=  6.31878D+01    |proj g|=  4.17255D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****    100    227      1     0     0   4.173D+01   6.319D+01
  F =   63.187813113493334     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/home/ran/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min finished


In [73]:
joblib.dump(model, 'TextClassifie-full-final.pkl')

['TextClassifie-full-final.pkl']

# 模型数据

In [122]:
print("模型方程: y = ", model.named_steps['logisticregression'].coef_, " * x + ", model.named_steps['logisticregression'].intercept_)

print("参数量: ", model.named_steps['logisticregression'].coef_.size + 1)

模型方程: y =  [[-5.66832414e-02  7.10293745e-03 -1.19980725e-03 ... -3.05276294e-04
  -1.54051859e-05 -1.10037042e-05]]  * x +  [-0.43367175]
参数量:  6687569


In [123]:
count_vectorizer = model.steps[0][1]

In [124]:
count_vectorizer.vocabulary_

{'年': 3010659,
 '江西': 4374078,
 '公务员': 1365833,
 '考试': 5414032,
 '行测': 5666562,
 '真题': 4956778,
 '及': 1890775,
 '参考答案': 1884268,
 '第一': 5123019,
 '部分': 6294966,
 '常识': 2989426,
 '判断': 1617338,
 '根据': 4193242,
 '题目': 6599490,
 '要求': 5724405,
 '在': 2332364,
 '四个': 2252855,
 '选项': 6225825,
 '中': 626135,
 '选出': 6218668,
 '一个': 288508,
 '最': 3983758,
 '恰当': 3285544,
 '的': 4815003,
 '答案': 5168373,
 '洲际导弹': 4436892,
 '通常': 6235933,
 '指': 3555839,
 '射程': 2811620,
 '大于': 2507753,
 '公里': 1383043,
 '远程': 6190309,
 '弹道式': 3134279,
 '导弹': 2805580,
 '目前': 4900323,
 '中国': 644647,
 '研制': 4990873,
 '洲际': 4436883,
 '弹道导弹': 3134267,
 '主': 699998,
 '要是': 5723975,
 '什么': 956963,
 '系列': 5226649,
 'a': 0,
 '东风': 589994,
 'b': 27423,
 '长征': 6407553,
 'c': 53561,
 '红旗': 5253221,
 'd': 88817,
 '天宫': 2541390,
 '关于': 1401344,
 '当代文学': 3151163,
 '下列': 461610,
 '说法': 5909368,
 '错误': 6393960,
 '是': 3913831,
 '白鹿原': 4809641,
 '获得': 5603415,
 '茅盾文学奖': 5589564,
 '平凡': 2998339,
 '世界': 572477,
 '史铁生': 2030009,
 '作品': 115

# Test

In [125]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from tabulate import tabulate

In [126]:
def predict_with_threshold(model, X, threshold=0.5):
    probabilities = model.predict_proba(X)
    
    positive_probabilities = probabilities[:, 1]

    predictions = (positive_probabilities > threshold).astype(int)
    
    return predictions, positive_probabilities

In [127]:
# test_dataset = datadict["test"]
test_dataset = validation_dataset

In [128]:
test_dataset = test_dataset.map(dataset_map_pre_process)

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [129]:
import time

start_time = time.time()

predictions, positive_probabilities = predict_with_threshold(model, test_dataset["text"])

print(f"time { time.time() - start_time}")

time 21.56321430206299


In [130]:
accuracy = accuracy_score(test_dataset["label"], predictions)
recall = recall_score(test_dataset["label"], predictions)
precision = precision_score(test_dataset["label"], predictions)
f1 = f1_score(test_dataset["label"], predictions)

In [131]:
data = [["Accuracy", accuracy], ["Recall", recall], ["Precision", precision], ["F1 Score", f1]]

print(tabulate(data, headers=["Metric", "Score"]))

Metric        Score
---------  --------
Accuracy   0.984298
Recall     0.977208
Precision  0.977208
F1 Score   0.977208


# Check

In [110]:
# 按照0.5threshold判断是否为试卷
# 文件路径为空则是当初数据集版本没有添加这个字段
for index, prediction in enumerate(predictions):
    if test_dataset['label'][index] != prediction:
        scope = '{:.2f}'.format(positive_probabilities[index])
        print(f"{index} -- {scope} -- {test_dataset['file_path'][index]}")

183 -- 0.93 -- negative_file/contents of B0709、历史的观念.doc
185 -- 0.01 -- positive_file/yxzse_cc_jy0501.doc
244 -- 0.46 -- positive_file/2018年清华大学自主招生考试模拟考试物理试题.docx
292 -- 0.65 -- negative_file/第14讲：公众号的运营及变现方式.docx
347 -- 0.12 -- 
394 -- 0.83 -- negative_file/三口第81课课前预习.doc
395 -- 1.00 -- 
462 -- 0.38 -- 
527 -- 0.37 -- 
561 -- 0.81 -- negative_file/党建理论知识应知应会【唯一微信 aoling18031988287】.doc
608 -- 0.00 -- positive_file/2010年中央机关遴选公务员考试真题及答案【官方抖音号：资深秘书】.docx
644 -- 0.23 -- 
650 -- 1.00 -- negative_file/瑞文智商测试解题技巧.doc
686 -- 0.37 -- 
759 -- 1.00 -- negative_file/党史党建知识题库100题【官方抖音号：材料大师姐】.docx
831 -- 0.56 -- negative_file/第二讲 舌苔 笔记.docx
834 -- 0.49 -- 
945 -- 0.46 -- positive_file/【公众号：zsxx_xxyg】2017年9月公共英语三级（pets3）口试模拟试题.docx
968 -- 0.00 -- positive_file/2013年江西南昌市遴选基层公务员考试真题及答案【官方抖音号：资深秘书】.docx


In [113]:
for line in test_dataset["text"][968].splitlines():
    print(line)

2013年江西南昌市遴选基层公务员考试真题及答案
一、简答题
1、行政决策的原则？
【参考答案】
第一，信息性原则；第二，预测原则；第三，系统性原则；第四，可行原则；第五，择优原则；第六，动态原则。
2、行政管理的特点？
【参考答案】
第一，一切行政活动都是直接或间接与国家权力相联系，以国家权力为基础的；
第二，行政管理是根据国家法律推行政务的组织活动；
第三，行政管理既管理社会的公共事务，又执行阶级统治的政治职能；
第四，行政管理要讲究管理的效能和效率；
第五，行政管理是人类改造社会的实践活动。
3、行政环境的内涵？
【参考答案】
第一，行政环境是针对具体行政系统而存在的；
第二，行政环境通过边界与行政系统相区分；
第三，行政环境构成成分的关键属性在于能够对行政系统的存在、运行与发展产生影响；
第四，行政环境因素既包括有形的事物，也包括无形的情势。
4、收文的主要程序？
【参考答案】
收文一般包括公文的传递、签收、登记、分办、拟办、承办、催办、办结、立卷、归档和销毁等程序。
二、论述题
1、行政监督的意义？
【参考答案】
行政监督的意义主要有：
第一，维护正常的行政秩序。
行政监督是惩罚行政机关及其工作人员违法、违规、违纪行为，保持正常行政秩序和良性行政行为的重要手段。为了保证整个政府机器正常运作、政令畅通和行政人员的廉洁奉公、遵纪守法，行政机关在法定的范围内恰当地行使职责，对各级各类行政机关的职权范围、工作规范、行政纪律等，必须作出明确的规定，并制定针对行政人员的贪污受贿、索贿、行贿、渎职等行为的一系列规章制度。如果没有这些法律和规章，政府行政行为就难以实现政令畅通、遵纪守法，行政活动就会产生混乱，整个政府机器和国家就会出现运转失常甚至瘫痪的现象。
第二，保护国家整体利益。
行政监督是保护国家整体利益和实现国家行政目标的客观要求。任何一个国家政权为了有效地实现其政治统治，必须尽力维护该政权的整体利益，因此，需要制定相关的法律、法规，采取多种监督制度和监督形式，确保国家整体利益不受损害。行政机关是国家利益的直接代表和维护者，行政机关及其工作人员的行为直接影响国家的整体利益。如果缺乏有效的行政监督机制，政府的各项行政法令和行政目标都难以顺利贯彻实施。
第三，提高行政工作效率。
行政监督是提高行政工作效率和实现科学管理的有效途径。行政监督本质上讲不是为了防范行政活动中